----

# 신경망 분석

인공 신경망 모델을 통해 ROC 분석을 실시한다.
- Predictive Analytics: NeuralNet, Bayesian, SVM, KNN : http://horicky.blogspot.com/2012/06/predictive-analytics-neuralnet-bayesian.html

- R - neuralnet 함수를 사용하여 역전파 알고리즘 사용 예제 : https://note.espriter.net/1263

----

## 1. Load Library

필요한 라이브러리와 데이터를 불러온다. 패키지가 없을 경우 설치해준다.

In [1]:
library(neuralnet)
library(nnet)
data(iris)
set.seed(67)  # 시드 설정. 동일한 결과가 나올 수 있도록 설정

## 2. Split Data

아이리스 데이터를 불러와 70%의 훈련용 데이터, 30% 검증용 데이터로 나눈다.

In [2]:
ind <- sample(2, nrow(iris), replace=T, prob=c(0.7, 0.4))
ind

[1] 2 1 1 2 1 1 1 1 1 2 1 1 1 1 2 1 2 1 1 1 1 2 2 1 2 2 1 1 2 1 1 2 1 1 1 1 1
 [38] 1 1 1 1 1 2 1 2 1 2 1 2 1 1 1 1 2 2 2 1 1 1 2 2 2 1 2 1 1 1 1 1 1 2 2 1 1
 [75] 2 1 2 1 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 1 2 2 1 1 2 1 2 2
[112] 1 1 2 1 1 1 1 1 1 2 2 1 2 2 1 1 1 1 1 1 1 1 2 1 1 2 2 1 2 1 1 2 1 2 1 1 2
[149] 1 1

In [3]:
trainData <- iris[ind==1,]
testData <- iris[ind==2,]
nrow(trainData)   # 훈련 데이터는 102개
nrow(testData)    # 검증 데이터는 48개

[1] 102

[1] 48

In [4]:
nnet_iristrain <- trainData
head(nnet_iristrain)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa


아이리스 품종별로 비교하는 칼럼을 추가한 후 구분하기 쉽게 이름을 변경한다.

In [5]:
nnet_iristrain <- cbind(nnet_iristrain, trainData$Species == 'setosa')
nnet_iristrain <- cbind(nnet_iristrain, trainData$Species == 'versicolor')
nnet_iristrain <- cbind(nnet_iristrain, trainData$Species == 'virginica')
names(nnet_iristrain)[6] <- 'setosa'
names(nnet_iristrain)[7] <- 'versicolor'
names(nnet_iristrain)[8] <- 'virginica'

In [6]:
head(nnet_iristrain)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,setosa,versicolor,virginica
2,4.9,3.0,1.4,0.2,setosa,TRUE,FALSE,FALSE
3,4.7,3.2,1.3,0.2,setosa,TRUE,FALSE,FALSE
5,5.0,3.6,1.4,0.2,setosa,TRUE,FALSE,FALSE
6,5.4,3.9,1.7,0.4,setosa,TRUE,FALSE,FALSE
7,4.6,3.4,1.4,0.3,setosa,TRUE,FALSE,FALSE
8,5.0,3.4,1.5,0.2,setosa,TRUE,FALSE,FALSE


## 3. Build Model

전처리한 데이터를 통해 모델을 훈련시킨다.

In [7]:
nn <- neuralnet(setosa + versicolor + virginica ~ Sepal.Length + Sepal.Width + Petal.Length + Petal.Width, 
                data=nnet_iristrain, hidden=c(3))
summary(nn)

                    Length Class      Mode    
call                  4    -none-     call    
response            306    -none-     logical 
covariate           408    -none-     numeric 
model.list            2    -none-     list    
err.fct               1    -none-     function
act.fct               1    -none-     function
linear.output         1    -none-     logical 
data                  8    data.frame list    
net.result            1    -none-     list    
weights               1    -none-     list    
startweights          1    -none-     list    
generalized.weights   1    -none-     list    
result.matrix        30    -none-     numeric 

## 4. Visualization

훈련한 모델을 그래프로 그려본다.
- 해당 환경에서는 그래프 기능 작동 안함

In [9]:
plot(nn)

## 5. Predict

만든 모델을 통해 예측작업을 실시한다.

In [11]:
mypredict <- compute(nn, testData[-5])$net.result
head(mypredict)

1,1.000272138,-0.0002979962002,0.00003174470793
4,1.000097846,-0.0001224162087,0.00003046045092
10,1.000239216,-0.0002648305632,0.00003150212219
15,1.000281914,-0.0003078442372,0.00003181674011
17,1.000277872,-0.0003037722422,0.00003178695604
22,1.000251391,-0.0002770951774,0.00003159183012


maxidx 라는 새로운 함수를 생성한다. 입력값중 최대값과 같은지 비교하는 기능이다.
- **apply()** : 일괄 적용 기능
- apply(X, MARGIN, FUN, ...) 양식으로 작성
- **MARGIN** : 1은 행, 2는 열, c(1,2)는 행렬을 의미

In [12]:
maxidx <- function(tmp){
    return(which(tmp == max(tmp)))
}

In [16]:
idx <- apply(mypredict, c(1), maxidx)
head(idx)  # 결과값에서 1은 첫번째 인덱스 값이 제일 크다는 의미

1  4 10 15 17 22 
 1  1  1  1  1  1

In [18]:
prediction <- c('setosa', 'verginica', 'versicolor')[idx]
prediction

[1] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [6] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
[11] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
[16] "verginica"  "verginica"  "verginica"  "verginica"  "verginica" 
[21] "verginica"  "verginica"  "verginica"  "verginica"  "verginica" 
[26] "verginica"  "verginica"  "verginica"  "verginica"  "versicolor"
[31] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
[36] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
[41] "versicolor" "verginica"  "versicolor" "versicolor" "versicolor"
[46] "versicolor" "versicolor" "versicolor"

## 6. Result

최종 결과를 분석해본다.

In [19]:
table(prediction, testData$Species)

            
prediction   setosa versicolor virginica
  setosa         15          0         0
  verginica       0         14         1
  versicolor      0          0        18

- 각 품종 별로 잘 분리되었음을 확인

----